In [13]:
import sys
sys.path.append('..')

import os
import json

import pandas as pd

In [14]:
datasets_path = os.path.join('..', 'datasets')

In [77]:
# Big color dataset
dpath_big = os.path.join(datasets_path, 'color-dict-big.json')
with open(dpath_big, 'r') as x:
    big = json.load(x)

In [78]:
# Standardize big color dataset
big_colors = pd.io.json.json_normalize(list(big['colors']))
big_colors = big_colors.drop(columns=['hex', 'luminance'])
big_colors = big_colors.rename(columns={'rgb.r': 'r', 'rgb.g': 'g', 'rgb.b': 'b'})

In [79]:
# Small color dataset
dpath_small = os.path.join(datasets_path, 'color-dict-small.json')
with open(dpath_small, 'r') as x:
    small = json.load(x)

In [103]:
# Standardize small color dataset
small_colors = []
for colorPage in small['colorPage']:
    for entry in colorPage['colorEntry']:
        small_colors.append(entry)
small_colors = pd.io.json.json_normalize(list(small_colors))
small_colors = small_colors.rename(columns={
    'colorName': 'name', 'RGB8.red': 'r', 
    'RGB8.green': 'g', 'RGB8.blue': 'b'})
small_colors['name'] = small_colors['name'].apply(lambda n: re.sub('\(SW.*\)', '', n).strip())